[View in Colaboratory](https://colab.research.google.com/github/clear21/Image-Quiz/blob/add-new-file/cifar10_20180911.ipynb)

In [1]:
!pip install keras

In [2]:
import keras
from keras.datasets import cifar10
from keras.utils import to_categorical 
from keras.layers import Input , Conv2D , MaxPooling2D , Flatten , Dense , Dropout , BatchNormalization
from keras.models import Model
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras.losses import categorical_crossentropy
from keras.initializers import RandomNormal
import numpy as np
import os

from keras.models import load_model

from google.colab import files

Using TensorFlow backend.


In [0]:
#ラベル
cifar10_labels = np.array([
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'])

#対象ラベルの番号のリスト生成
#引数：対象ラベル名のリスト　、　全ラベル名のリスト（デフォルト設定あり）
#戻り値：第二引数における第一引数それぞれのラベル名のインデックスを要素とするリスト
def create_gen_labels_num(gen_labels , cifar10_labels = cifar10_labels):
    gen_labels_num = []
  
    for i in range(len(cifar10_labels)):
        if cifar10_labels[i] in gen_labels:          
            gen_labels_num.append(i)
    
    return gen_labels_num


#学習・検証データ準備
#引数：生成するデータのラベル名リスト
def gen_data(gen_labels):
    #全データ（～_all）
    (x_train_tmp_all, y_train_tmp_all) , (x_test_tmp_all, y_test_tmp_all) = cifar10.load_data()
    
    #対象ラベルの番号（yの値(0～9)と対応）
    gen_labels_num = create_gen_labels_num(gen_labels)
    print('gen_labels:' , gen_labels)
    print('gen_labels_num:' , gen_labels_num)
    
    #生成対象ラベルのデータのみに絞る
    
    #--学習データ
    y_train_tmp_enum = list(filter(lambda x : x[1] in gen_labels_num, enumerate(y_train_tmp_all)))
    y_train_tmp_bef_trans = np.array([x[1] for x in y_train_tmp_enum])
    
    taisho_index_list_train = [x[0] for x in y_train_tmp_enum]
    
    x_train_tmp_enum = list(filter(lambda x : x[0] in taisho_index_list_train, enumerate(x_train_tmp_all)))
    x_train_tmp = np.array([x[1] for x in x_train_tmp_enum])
    
    #--検証データ
    y_test_tmp_enum = list(filter(lambda x : x[1] in gen_labels_num, enumerate(y_test_tmp_all)))
    y_test_tmp_bef_trans = np.array([x[1] for x in y_test_tmp_enum])
    
    taisho_index_list_test = [x[0] for x in y_test_tmp_enum]
    
    x_test_tmp_enum = list(filter(lambda x : x[0] in taisho_index_list_test, enumerate(x_test_tmp_all)))
    x_test_tmp = np.array([x[1] for x in x_test_tmp_enum])
    
    #ラベルを連番に振りなおす
    transform_labels_num_dict = {}
    ato_labels_num = 0
    
    for mae_labels_num in gen_labels_num:
        transform_labels_num_dict[mae_labels_num] = ato_labels_num
        ato_labels_num += 1
    
    y_train_tmp = np.array(list(map(lambda x : [transform_labels_num_dict[x[0]]] , y_train_tmp_bef_trans)))
    y_test_tmp = np.array(list(map(lambda x : [transform_labels_num_dict[x[0]]] , y_test_tmp_bef_trans)))
    
    #データを0～1にする。
    x_train = x_train_tmp.astype('float32') / 255
    x_test = x_test_tmp.astype('float32') / 255
    
    #y_train,y_testをone_hotにする。
    y_train = to_categorical(y_train_tmp , len(gen_labels))
    y_test = to_categorical(y_test_tmp , len(gen_labels))
    
    return (x_train , y_train) , (x_test , y_test)
  
#モデル
def CNN_model_for_cifar10(class_num , input_shape=(32 , 32 , 3)):
    
    #■sigmoid初期化（xavier）
    #Conv2D(128 , kernel_size , padding = 'same' , activation  = 'sigmoid' , kernel_initializer = RandomNormal(stddev=(1/np.sqrt(128))))

    #■relu初期化（he）
    #Conv2D(128 , kernel_size , padding = 'same' , activation  = 'relu' , kernel_initializer = RandomNormal(stddev=(2/np.sqrt(128))))
    
    input = Input(input_shape)
    
    kernel_size = (3 , 3)
    max_pool_size = (2 , 2)

    #【sameなので32×32のまま】32 × 32 → (32-3+1) × (32-3+1) = 30 × 30
    cnn = Conv2D(32 , kernel_size , padding = 'same' , activation  = 'relu')(input)
    cnn = Conv2D(32 , kernel_size , padding = 'valid' , activation  = 'relu')(cnn)
    cnn = MaxPooling2D(pool_size = max_pool_size , padding = 'same')(cnn)
    cnn = Dropout(0.25)(cnn)

    cnn = Conv2D(32 , kernel_size , padding = 'same' , activation  = 'relu')(input)
    cnn = Conv2D(32 , kernel_size , padding = 'valid' , activation  = 'relu')(cnn)
    cnn = MaxPooling2D(pool_size = max_pool_size , padding = 'same')(cnn)
    cnn = Dropout(0.25)(cnn)
    
    cnn = Conv2D(64 , kernel_size , padding = 'same' , activation  = 'relu')(cnn)
    cnn = Conv2D(64 , kernel_size , activation  = 'relu')(cnn)
    cnn = MaxPooling2D(pool_size = max_pool_size)(cnn)
    cnn = Dropout(0.25)(cnn)
    
    cnn = Conv2D(64 , kernel_size , padding = 'same' , activation  = 'relu')(cnn)
    cnn = Conv2D(64 , kernel_size , activation  = 'relu')(cnn)
    cnn = MaxPooling2D(pool_size = max_pool_size)(cnn)
    cnn = Dropout(0.25)(cnn)

    fc = Flatten()(cnn)
    
    fc = Dense(512 , activation = 'relu')(fc)
    fc = Dropout(0.5)(fc)
    softmax = Dense(class_num , activation = 'softmax')(fc)
    
    model = Model(input=input , output=softmax)
    
    return model
  
#コールバック関数（モデル保存）
#※変数 taisho_labels_text が既存であることが前提  --未設定
class ModelSave(Callback):
  
    #def __init__(self , output_dir_path):
    def __init__(self):
        #self.output_dir_path = output_dir_path
        super().__init__()
    
    def on_epoch_end(self , epoch , logs=None):
        #if not os.path.isdir(self.output_dir_path):
            #os.makedirs(output_dir_path)
        
        #モデルを保存
        if (epoch + 1) == epochs :
            weight_file_name = 'model_'+ taisho_labels_text + '.hdf5'.format(epoch + 1)
            #weight_file_path = os.path.join(self.output_dir_path , weight_file_name)
            
            #self.model.save(weight_file_path)
            self.model.save(weight_file_name)
            
            #print('Saved trained model at %s ' % weight_file_path)
            print('Saved trained model：' , weight_file_name)

#検証
def predict_accuracy(x_batch , y_batch , model):
    preds = model.predict(x_batch , verbose = 1)
    
    batch_size = len(x_batch)
    
    count = 0
    
    for (y , pred)  in zip(y_batch , preds):
        if np.argmax(y) == np.argmax(pred):
            count += 1
    print('acc' , round(count / batch_size * 100 , 2) , '%' )

In [8]:
gen_labels_list = [
    [cifar10_labels[3] , cifar10_labels[5]] #dog , cat
    ,[cifar10_labels[0] , cifar10_labels[2]] #airplane , bird
    ,[cifar10_labels[4] , cifar10_labels[7]] #deer , hourse
    ,[cifar10_labels[0] , cifar10_labels[1] , cifar10_labels[8] , cifar10_labels[9]] #乗り物
    ,[cifar10_labels[i] for i in [2,3,4,5,6,7]] #生き物
    ,cifar10_labels
]

model_file_name_list = []

for gen_labels in gen_labels_list:
    epochs = 40

    gen_labels_num = create_gen_labels_num(gen_labels)
    
    taisho_labels_text = '_'.join(list(map(lambda x : str(x) , gen_labels_num)))
    
    model_path = 'model_'+ taisho_labels_text + '.hdf5'
    
    print('*' * 30)
    print('対象ラベル：' , gen_labels)
    print('taisho_labels_text : ' , taisho_labels_text)
    print('model_path : ' , model_path)
    
    (x_train , y_train) , (x_test , y_test) = gen_data(gen_labels)

    #学習
    model = CNN_model_for_cifar10(len(gen_labels))
    model.compile(loss=categorical_crossentropy , optimizer='SGD' , metrics=['accuracy'])
    #model.compile(loss=categorical_crossentropy , optimizer=SGD(lr=0.01 , momentum=0.9 , nesterov=True))
    #--コールバック生成
    callback = ModelSave()
    model.fit(x_train, y_train , epochs = epochs , batch_size = 50 , callbacks = [callback])
    
    model_file_name_list.append(model_path)
    
    #検証
    pred_model = load_model(model_path) 
    predict_accuracy(x_test, y_test , pred_model)

******************************
対象ラベル： ['cat', 'dog']
taisho_labels_text :  3_5
model_path :  model_3_5.hdf5
170500096/170498071 [==============================] - 45s 0us/step
gen_labels: ['cat', 'dog']
gen_labels_num: [3, 5]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:119: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/40
10000/10000 [==============================] - 6s 587us/step - loss: 0.6946 - acc: 0.5023
Epoch 2/40
10000/10000 [==============================] - 4s 369us/step - loss: 0.6927 - acc: 0.5093
Epoch 3/40
10000/10000 [==============================] - 4s 364us/step - loss: 0.6921 - acc: 0.5151
Epoch 4/40
10000/10000 [==============================] - 4s 363us/step - loss: 0.6922 - acc: 0.5233
Epoch 5/40
10000/10000 [==============================] - 4s 357us/step - loss: 0.6908 - acc: 0.5286
Epoch 6/40
10000/10000 [==============================] - 4s 357us/step - loss: 0.6901 - acc: 0.5310
Epoch 7/40
10000/10000 [==============================] - 4s 357us/step - loss: 0.6884 - acc: 0.5461
Epoch 8/40
10000/10000 [==============================] - 4s 357us/step - loss: 0.6878 - acc: 0.5444
Epoch 9/40
10000/10000 [==============================] - 3s 349us/step - loss: 0.6866 - acc: 0.5486
Epoch 10/40
10000/10000 [==============================] - 4s 352us/step - loss: 0.6852 - a

In [0]:
#モデルのダウンロード
for fname in model_file_name_list:
    files.download(fname)